In [1]:
import torchvision
from __future__ import print_function, division
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch
import torch.nn.functional as F
import os
from tqdm import tqdm

from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision.models import vgg16
from sklearn.metrics import accuracy_score

In [2]:
class ImageEmotionModel(nn.Module):
    def __init__(self, num_classes=6, feature_extractor=vgg16):
        super(ImageEmotionModel, self).__init__()
        # 使用预训练模型作为特征提取器
        self.feature_extractor = feature_extractor(pretrained=True)
        # 需要注意的是，这里可能会冻结部分层不参与训练
        for param in self.feature_extractor.parameters():
            param.requires_grad = False # 或者只冻结某些层

        # 添加新的全连接层用于情感分类
        self.classifier = nn.Sequential(
            nn.Linear(25088, 4096),
            nn.ReLU(),
            nn.Linear(4096, 1000),
            nn.ReLU(),
            nn.Linear(1000, num_classes),
            )

    def forward(self, x):
        x=self.feature_extractor.features(x)
        x=nn.Flatten(-3)(x)
        x=self.classifier(x)
        return x


In [3]:
train_transform = transforms.Compose([
        transforms.Resize(250),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

full_dataset =torchvision.datasets.ImageFolder(root='images',transform=train_transform)

# length 数据集总长度
full_data_size = len(full_dataset)
print("总数据集的长度为：{}".format(full_data_size))

train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(
    full_dataset, [train_size, test_size])

总数据集的长度为：1980


In [4]:
# torch.save(train_dataset, 'train_dataset1.pkl')
# torch.save(test_dataset, 'test_dataset1.pkl')

In [5]:
train_dataset=torch.load('train_dataset1.pkl')
test_dataset=torch.load('test_dataset1.pkl')

train_loader = DataLoader(dataset=train_dataset, batch_size=16,
                          shuffle=True, num_workers=0, drop_last=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=5,
                         shuffle=True, num_workers=0, drop_last=True)

train_data_size = len(train_loader)
test_data_size = len(test_loader)
print(train_data_size, test_data_size)

99 79


In [6]:
Epoch_path = "Epoch1.txt"

def get_epoch(path):
    if not os.path.exists(path): 
        Epoch_file=open(path, "w")
        Epoch_file.write("-1")
        Epoch_file.close()
    Epoch_file=open(path, "r")
    s=Epoch_file.readline()
    Epoch_file.close()
    return int(s)+1

def set_epoch(path):
    if not os.path.exists(path): 
        Epoch_file=open(path, "w")
        Epoch_file.write("-1")
        Epoch_file.close()
    Epoch_file=open(path, "r")
    s=Epoch_file.readline()
    Epoch_file.close()
    Epoch_file=open(path, "w")
    Epoch_file.write(str(int(s)+1))
    Epoch_file.close()
# print(get_epoch("Epoch.txt"))
# set_epoch("Epoch.txt")
get_epoch(Epoch_path)

81

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model= ImageEmotionModel()
model.to(device)
model.train()
model_test = ImageEmotionModel()
model_test.to(device)
model_test.eval()
pass

c:\software\ai\conda\envs\310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\software\ai\conda\envs\310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=False, threshold=0.00001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
ture_epoch = get_epoch(Epoch_path)
if ture_epoch > 0:
    model.load_state_dict(torch.load(f'weights1/{ture_epoch-1}_model.pth'))
    optimizer.load_state_dict(torch.load(
        f'weights1/{ture_epoch-1}_optimizer.pth'))
loss_function = nn.CrossEntropyLoss()



Epoch = 40
for epoch in tqdm(range(Epoch), desc="train",unit='epoch'):
    loss_all=None
    log_loss = open("log1.csv", "a")
    ture_epoch = get_epoch(Epoch_path)
    loss_sum = 0
    st = 0
    for x, y in train_loader: 
        optimizer.zero_grad()
        x = x.to(device)
        y = y.type(torch.long).to(device)
        out= model(x)
        loss = loss_function(out, y)
        if loss_all is None:
            loss_all = loss
        else:
            loss_all += loss
        loss.backward()
        optimizer.step()
        st += 1
        loss_sum += loss.to("cpu").detach().numpy()

    
    log_loss.write("{}, {}\n".format(ture_epoch, loss_sum/st))

    name = f"weights1/{ture_epoch}"
    if (epoch+1) % 1 == 0:
        torch.save(model.state_dict(), name+"_model.pth")
        torch.save(optimizer.state_dict(), name+"_optimizer.pth")
        # email_remind.send_email(f"{ture_epoch}轮训练完成")
    # print("epoch:{},loss:{}".format(epoch, loss_sum/st))
    log_loss.close()
    set_epoch(Epoch_path)  # ture_epoch ++
    scheduler.step(loss_all)



    if (epoch+1) % 1== 0:
        # 评估
        ture_epoch = get_epoch(Epoch_path)
        if ture_epoch > 0:
            model_test.load_state_dict(torch.load(
                f'weights1/{ture_epoch-1}_model.pth'))
        acc=0
        for x, y in test_loader: 
            out= model_test(x.to(device))
            y_pred=out.to("cpu").detach().numpy()
            y_true=y.to("cpu").detach().numpy()
            ac1=accuracy_score(y_true, y_pred.argmax(axis=-1))
            acc+=ac1
        acc=acc/test_loader.__len__()
        acc_loss = open("acc1.csv", "a")
        acc_loss.write("{}, {}\n".format(get_epoch(Epoch_path), acc))
        acc_loss.close()
#         email_remind.send_email(f"acc={acc}")
# email_remind.send_email(None)

train: 100%|██████████| 40/40 [15:13<00:00, 22.85s/epoch]


In [9]:
from sklearn.metrics import accuracy_score
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_test = ImageEmotionModel()

# ture_epoch = get_epoch(Epoch_path)
# if ture_epoch > 0:
#     model_test.load_state_dict(torch.load(
#         f'weights/{ture_epoch-1}_model.pth'))

model_test.load_state_dict(torch.load(
        f'weights1/{80}_model.pth'))

model_test.to(device)
model_test.eval()
acc=0
for x, y in tqdm(test_loader): 
    out= model_test(x.to(device))
    y_pred=out.to("cpu").detach().numpy()
    y_true=y.to("cpu").detach().numpy()
    ac1=accuracy_score(y_true, y_pred.argmax(axis=-1))
    acc+=ac1
acc/test_loader.__len__()


c:\software\ai\conda\envs\310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\software\ai\conda\envs\310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 79/79 [00:04<00:00, 19.20it/s]


0.38481012658227826

In [10]:
l=("愤怒","厌恶","恐惧","快乐","悲伤","惊喜")